In [1]:
import os
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import DocArrayInMemorySearch
from operator import itemgetter

/Users/jdartland/Developer/ragapp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()

MODEL = os.getenv("OLLAMA_MODEL")

try:
    print(f"Trying to load model {MODEL} from local Ollama instance")
    embeddings = OllamaEmbeddings(model=MODEL)
    model = Ollama(model=MODEL)
    model.invoke("")
except Exception as e:
    print(f"Could not load model {MODEL} from local Ollama instance. Exception: {e}")
    exit(1)

Trying to load model codellama from local Ollama instance


In [3]:
# Load a textfile, split it into chunks and create a local in-memory vectorstore with the LLM embeddings
loader = TextLoader("examplelog.txt")
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
input = loader.load()
documents = text_splitter.split_documents(input)
vectorstore = DocArrayInMemorySearch.from_documents(documents, embeddings)



Created a chunk of size 9401, which is longer than the specified 1000
Created a chunk of size 2306, which is longer than the specified 1000
Created a chunk of size 6568, which is longer than the specified 1000
Created a chunk of size 2306, which is longer than the specified 1000
Created a chunk of size 6317, which is longer than the specified 1000
/Users/jdartland/Developer/ragapp/.venv/lib/python3.9/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [4]:
# Create a retiever component from the vectorstore - a retriver is something that gets any infromation(not only from a vectorstore)
retriver = vectorstore.as_retriever()
# Return the 4 top most relevant documents which relates to the word "ERROR"
retriver.invoke("ERROR")

[Document(page_content="The last packet sent successfully to the server was 0 milliseconds ago. The driver has not received any packets from the server.\n\tat java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)\n\tat java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)\n\tat java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)\n\tat java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)\n\tat java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)\n\tat com.mysql.cj.exceptions.ExceptionFactory.createException(ExceptionFactory.java:61)\n\tat com.mysql.cj.exceptions.ExceptionFactory.createException(ExceptionFactory.java:105)\n\tat com.mysql.cj.exceptions.ExceptionFactory.createException(ExceptionFactory.java:151)\n\tat com.mysql.cj.exceptions.ExceptionFactory.createCommunicationsEx

In [5]:
# Define a prompt template which will be used to generate the final response
template = """Answer the following questions in as much detail as possible. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

"""

In [6]:
prompt = PromptTemplate.from_template(template)
prompt.format(context="Some input", question="Tell me about the latest 3 errors in the Log.txt file")

parser = StrOutputParser()

# create a chain - chain is a pipeline of prompt(expect a runnable map containing a context and a question), model and parser
# context - pass the question to a retriver to obtain the context
# question - the value of the question
chain = ( {
    "context": itemgetter("question") | retriver, 
    "question": itemgetter("question")
    } 
    | prompt | model | parser)

chain.input_schema.schema()

{'title': 'RunnableParallel<context,question>Input',
 'type': 'object',
 'properties': {'question': {'title': 'Question'}}}

In [8]:
questions = [
    "Tell me about the latest 3 errors in the Log.txt file",
    "At what time did the latest error occur?",
    "Give me a suggestion on how to fix it",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: Tell me about the latest 3 errors in the Log.txt file
Answer:  The last three errors in the `Log.txt` file are as follows:

1. `com.mysql.cj.exceptions.CJCommunicationsException`: This exception indicates a problem with the communication between your Spring Boot application and the MySQL database. It suggests that there is a connection issue, such as the database server being down or not responding to requests.
2. `java.net.ConnectException: Connection refused (Connection refused)`: This exception indicates that the connection to the database server was refused, likely due to firewall rules or incorrect network configuration. Make sure that your Spring Boot application can connect to the MySQL database and that there are no issues with the network configuration.
3. `Caused by: com.mysql.cj.jdbc.exceptions.CommunicationsException: Communications link failure`: This exception is a more detailed version of the first one, indicating that there is still an issue with the communica